<br>
<br>

# <strong>AATCV.<strong>
### DETECTION STREAMING EXPERIMENTATION:

<br>
<br>

#### 3.01 ENVIRONMENT SET UP.

- Firstly, this notebook must be run in Google Colab with a GPU.
- Create a file in your Google Drive called yolov4-tiny and insert into it your config file, your trained weights, your .names file, .data file, train.txt, and test.txt.
- Once opened in Colab, run the cells below to clone the darknet repo into the Colab environment, mount Google Drive, and alter/run the makefile to build darknet.

In [1]:
#To clone darknet repo into environment:
!git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 15420, done.
remote: Total 15420 (delta 0), reused 0 (delta 0), pack-reused 15420
Receiving objects: 100% (15420/15420), 14.02 MiB | 13.73 MiB/s, done.
Resolving deltas: 100% (10362/10362), done.


In [2]:
#To mount drive, create symbolic link of /mydrive, and list contents of mounted drive:
%cd ..
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive/yolov4-tiny

/
Mounted at /content/gdrive
new_custom-yolov4-tiny-detector_best.weights  obj.data	 test.txt     training
new_custom-yolov4-tiny-detector.cfg	      obj.names  test_videos  train.txt


In [3]:
# To enable GPU, OPENCV, CUDNN, CUDNN_HALF, and LIBSO within makefile by switching 0 to 1:
%cd /content/darknet/
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!sed -i 's/CUDNN_HALF=0/CUDNN_HALF=1/' Makefile
!sed -i 's/LIBSO=0/LIBSO=1/' Makefile

/content/darknet


In [4]:
# Build darknet using makefile: 
!make

mkdir -p ./obj/
mkdir -p backup
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -DCUDNN_HALF -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -DCUDNN_HALF -fPIC -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:946:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void draw_train_loss(char*, void**, int, float, float, int, int, float, int, char*, float, int, int, double)’:
./src/image_opencv.cpp:1147:13: warning: this ‘if’ clause does not guard... [-Wmisleading-indentation]
             if (iteration_old ==

<br>
<br>

#### 3.02 DIRECTORY CLEAN UP.

- Run the cells below to clear out the cfg/ and data/ directories.
- Run the following cell to copy the files from the mounted Google Drive into their respective directories.

In [5]:
# To clean data/ and cfg/ files:
%cd data/
!find -maxdepth 1 -type f -exec rm -rf {} \;
%cd ..

%rm -rf cfg/
%mkdir cfg

/content/darknet/data
/content/darknet


In [6]:
#To copy necessary files from drive to respective directories:
!cp /mydrive/yolov4-tiny/new_custom-yolov4-tiny-detector_best.weights ./cfg
!cp /mydrive/yolov4-tiny/new_custom-yolov4-tiny-detector.cfg ./cfg
!cp /mydrive/yolov4-tiny/obj.names ./data
!cp /mydrive/yolov4-tiny/obj.data  ./data
!cp /mydrive/yolov4-tiny/train.txt ./data
!cp /mydrive/yolov4-tiny/test.txt  ./data

<br>
<br>

#### 3.03 STREAMING DETECTION FROM WEBCAM.

- Run the cell below to open a frame that will show live video streaming from webcam with live object detection overlaid. It works!

In [7]:
ls

3rdparty/               darknet_video.py        obj/
backup/                 data/                   README.md
build/                  image_yolov3.sh*        results/
build.ps1*              image_yolov4.sh*        scripts/
cfg/                    include/                src/
cmake/                  json_mjpeg_streams.sh*  uselib*
CMakeLists.txt          libdarknet.so*          vcpkg.json
darknet*                LICENSE                 video_yolov3.sh*
DarknetConfig.cmake.in  Makefile                video_yolov4.sh*
darknet_images.py       net_cam_v3.sh*
darknet.py              net_cam_v4.sh*


In [79]:
# Customized code incorporating code from https://github.com/theAIGuysCode/YOLOv4-Cloud-Tutorial/blob/master/yolov4_webcam.ipynb:
from IPython.display import display, Javascript, Image
from google.colab.output import eval_js
from google.colab.patches import cv2_imshow
from base64 import b64decode, b64encode
import cv2
import numpy as np
import PIL
import io
import html
import time
from statistics import mean
from datetime import datetime
import matplotlib.pyplot as plt
%matplotlib inline

from darknet import *
network, class_names, class_colors = load_network("cfg/new_custom-yolov4-tiny-detector.cfg", "data/obj.data", "cfg/new_custom-yolov4-tiny-detector_best.weights")
width = network_width(network)
height = network_height(network)

def darknet_helper(img, width, height):
  darknet_image = make_image(width, height, 3)
  img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  img_resized = cv2.resize(img_rgb, (width, height),
                              interpolation=cv2.INTER_LINEAR)

  img_height, img_width, _ = img.shape
  width_ratio = img_width/width
  height_ratio = img_height/height

  copy_image_from_bytes(darknet_image, img_resized.tobytes())
  detections = detect_image(network, class_names, darknet_image)
  free_image(darknet_image)
  return detections, width_ratio, height_ratio

def js_to_image(js_reply):
  """
  Params:
          js_reply: JavaScript object containing image from webcam
  Returns:
          img: OpenCV BGR image
  """
  image_bytes = b64decode(js_reply.split(',')[1])
  jpg_as_np = np.frombuffer(image_bytes, dtype=np.uint8)
  img = cv2.imdecode(jpg_as_np, flags=1)
  return img

def bbox_to_bytes(bbox_array):
  """
  Params:
          bbox_array: Numpy array (pixels) containing rectangle to overlay on video stream.
  Returns:
        bytes: Base64 image byte string
  """
  bbox_PIL = PIL.Image.fromarray(bbox_array, 'RGBA')
  iobuf = io.BytesIO()
  bbox_PIL.save(iobuf, format='png')
  bbox_bytes = 'data:image/png;base64,{}'.format((str(b64encode(iobuf.getvalue()), 'utf-8')))
  return bbox_bytes  

def midpoint(ptA, ptB):
    return (ptA[0], ((ptB[1] - ptA[1]) * 0.5) + ptA[1])
def midpoint_01(ptA, ptB):
    return (ptA[0], ((ptB[1] - ptA[1]) * 0.333) + ptA[1])
def midpoint_02(ptA, ptB):
    return (ptA[0], ((ptB[1] - ptA[1]) * 0.666) + ptA[1])
def midpoint2(ptA, ptB):
    return (((ptB[0] - ptA[0]) * 0.5) + ptA[0], ptA[1])
def midpoint2_01(ptA, ptB):
    return (((ptB[0] - ptA[0]) * 0.333) + ptA[0], ptA[1])
def midpoint2_02(ptA, ptB):
    return (((ptB[0] - ptA[0]) * 0.666) + ptA[0], ptA[1])

# Video Stream Java Script:
def video_stream():
  js = Javascript('''
    var video;
    var div = null;
    var stream;
    var captureCanvas;
    var imgElement;
    var labelElement;
    
    var pendingResolve = null;
    var shutdown = false;
    
    function removeDom() {
      stream.getVideoTracks()[0].stop(); //HEREEEEEEEEEEEEEEEEEEEEEEEEEEE
       video.remove();
       div.remove();
       video = null;
       div = null;
       stream = null;
       imgElement = null;
       captureCanvas = null;
       labelElement = null;
    }
    
    function onAnimationFrame() {
      if (!shutdown) {
        window.requestAnimationFrame(onAnimationFrame);
      }
      if (pendingResolve) {
        var result = "";
        if (!shutdown) {
          captureCanvas.getContext('2d').drawImage(video, 0, 0, 640, 480);
          result = captureCanvas.toDataURL('image/jpeg', 0.8)
        }
        var lp = pendingResolve;
        pendingResolve = null;
        lp(result);
      }
    }
    
    async function createDom() {
      if (div !== null) {
        return stream;
      }

      div = document.createElement('div');
      div.style.border = '2px solid black';
      div.style.padding = '3px';
      div.style.width = '100%';
      div.style.maxWidth = '1200px';
      document.body.appendChild(div);
      
      const modelOut = document.createElement('div');
      modelOut.innerHTML = "<span>Status:</span>";
      labelElement = document.createElement('span');
      labelElement.innerText = 'No data';
      labelElement.style.fontWeight = 'bold';
      modelOut.appendChild(labelElement);
      div.appendChild(modelOut);
           
      video = document.createElement('video');
      video.style.display = 'block';
      video.width = div.clientWidth - 6;
      video.setAttribute('playsinline', '');
      video.onclick = () => { shutdown = true; };
      stream = await navigator.mediaDevices.getUserMedia(
          {video: { facingMode: "environment"}});
      div.appendChild(video);

      imgElement = document.createElement('img');
      imgElement.style.position = 'absolute';
      imgElement.style.zIndex = 1;
      imgElement.onclick = () => { shutdown = true; };
      div.appendChild(imgElement);
      
      const instruction = document.createElement('div');
      instruction.innerHTML = 
          '<span style="color: red; font-weight: bold;">' +
          'When finished, click here or on the video to stop this demo</span>';
      div.appendChild(instruction);
      instruction.onclick = () => { shutdown = true; };
      
      video.srcObject = stream;
      await video.play();

      captureCanvas = document.createElement('canvas');
      captureCanvas.width = 640; //video.videoWidth;
      captureCanvas.height = 480; //video.videoHeight;
      window.requestAnimationFrame(onAnimationFrame);
      
      return stream;
    }
    async function stream_frame(label, imgData) {
      if (shutdown) {
        removeDom();
        shutdown = false;
        return '';
      }

      var preCreate = Date.now();
      stream = await createDom();
      
      var preShow = Date.now();
      if (label != "") {
        labelElement.innerHTML = label;
      }
            
      if (imgData != "") {
        var videoRect = video.getClientRects()[0]; //HEREEEEEEEEEEEEEEEEEEEEEEEEEEEEEEE
        imgElement.style.top = videoRect.top + "px";
        imgElement.style.left = videoRect.left + "px";
        imgElement.style.width = videoRect.width + "px";
        imgElement.style.height = videoRect.height + "px";
        imgElement.src = imgData;
      }
      
      var preCapture = Date.now();
      var result = await new Promise(function(resolve, reject) {
        pendingResolve = resolve;
      });
      shutdown = false;
      
      return {'create': preShow - preCreate, 
              'show': preCapture - preShow, 
              'capture': Date.now() - preCapture,
              'img': result};
    }
    ''')

  display(js)
  
def video_frame(label, bbox):
  data = eval_js('stream_frame("{}", "{}")'.format(label, bbox))
  return data

video_stream()
label_html = 'Capturing...'
bbox = ''
count = 0
legality = 0 
det_count = 1
legality_list = [legality]
det_list = [det_count]
og = time.perf_counter()
minute_count = 0
while True:
    elapsed = time.perf_counter() - og
    js_reply = video_frame(label_html, bbox)
    if not js_reply:
        break

    frame = js_to_image(js_reply["img"])
    bbox_array = np.zeros([480,640,4], dtype=np.uint8)
    detections, width_ratio, height_ratio = darknet_helper(frame, width, height)

#Detections Loop:
    craw_count = 0
    for label, confidence, bbox in detections:
      if float(confidence) > 80.0:
        det_count += 1
        craw_count += 1
      left, top, right, bottom = bbox2points(bbox)
      left, top, right, bottom = int(left * width_ratio), int(top * height_ratio), int(right * width_ratio), int(bottom * height_ratio)
      top_length = right - left
      left_length = bottom - top
      if left_length > top_length:
        length = (left_length * .3) * .015
        if length >= .8:
          legal = 'LEGAL'
          if float(confidence) > 80.0:
            legality += 1
        elif length < 1:
          legal = " "
      elif left_length < top_length:
        length = (top_length *.3) *.015
        if length >= .8:
          legal = 'LEGAL'
          if float(confidence) > 80.0:
            legality += 1
        elif length < 1:
          legal = " "
      bbox_array = cv2.rectangle(bbox_array, (left, top), (right, bottom), (255,255,255) , 2)
      (tl, tr, br, bl) = ((top, left), (top,right), (bottom,left), (bottom,right))
      (tltrX_01, tltrY_01) = midpoint_01(tl, tr)
      (blbrX_01, blbrY_01) = midpoint_01(bl, br)
      (tltrX_02, tltrY_02) = midpoint_02(tl, tr)
      (blbrX_02, blbrY_02) = midpoint_02(bl, br)
      (tlblX_01, tlblY_01) = midpoint2_01(tl, bl)
      (trbrX_01, trbrY_01) = midpoint2_01(tr, br)
      (tlblX_02, tlblY_02) = midpoint2_02(tl, bl)
      (trbrX_02, trbrY_02) = midpoint2_02(tr, br)
      cv2.line(bbox_array, (int(tltrY_01), int(tltrX_01)), (int(blbrY_02),int(blbrX_02)),(255, 255, 255), 1)
      cv2.line(bbox_array, (int(tltrY_02), int(tltrX_02)), (int(blbrY_01),int(blbrX_01)),(255, 255, 255), 1)
      cv2.line(bbox_array, (int(tlblY_01), int(tlblX_01)), (int(trbrY_01), int(trbrX_01)),(255, 255, 255), 1)
      cv2.line(bbox_array, (int(tlblY_02), int(tlblX_02)), (int(trbrY_02), int(trbrX_02)),(255, 255, 255), 1)
      bbox_array = cv2.putText(bbox_array, "{}".format(str(confidence) +'% ' + str(round(length,1)) + 'in.  ' + legal),
                        (left, top - 5), cv2.FONT_HERSHEY_DUPLEX, 0.5,
                        (255,255,255), 1)

    bbox_array = cv2.putText(bbox_array, "{}".format('CURRENT LEGAL PERCENTAGE: ' + str(round((legality/det_count)*100,1)) + '%'),
                        (10,20), cv2.FONT_HERSHEY_DUPLEX, 0.4,(255,255,255), 1)
    bbox_array = cv2.putText(bbox_array, "{}".format('AVG LEGAL PERCENTAGE: ' + str(round((mean(legality_list)/mean(det_list))*100,1)) + '%'),
                        (10,40), cv2.FONT_HERSHEY_DUPLEX, 0.4,(255,255,255), 1)  
    bbox_array = cv2.putText(bbox_array, "{}".format('CURRENT COUNT HIGH CONF.: ' + str(craw_count)),
                        (10,60), cv2.FONT_HERSHEY_DUPLEX, 0.4,(255,255,255), 1)
    bbox_array = cv2.putText(bbox_array, "{}".format('RECENT LEGAL DET: ' + str(legality)),
                        (10,80), cv2.FONT_HERSHEY_DUPLEX, .4,(255,255,255), 1) 
    bbox_array = cv2.putText(bbox_array, "{}".format('AVG LEGAL DET: ' + str(round(mean(legality_list)))),
                        (10,100), cv2.FONT_HERSHEY_DUPLEX, 0.4,(255,255,255), 1) 
    bbox_array = cv2.putText(bbox_array, "{}".format('SEC SINCE AVG: ' + str(round(elapsed))),
                        (10,420), cv2.FONT_HERSHEY_DUPLEX, 0.4,(255,255,255), 1)
    bbox_array = cv2.putText(bbox_array, "{}".format('MINUTES ELAPSED: ' + str(minute_count)),
                        (10,440), cv2.FONT_HERSHEY_DUPLEX, 0.4,(255,255,255), 1) 
    now = datetime.now()
    bbox_array = cv2.putText(bbox_array, "{}".format(str(now)),
                        (10,460), cv2.FONT_HERSHEY_DUPLEX, 0.4,(255,255,255), 1) 
    bbox_array[:,:,3] = (bbox_array.max(axis = 2) > 0 ).astype(int) * 255
    bbox_bytes = bbox_to_bytes(bbox_array)
    bbox = bbox_bytes 
    if elapsed >= 60:
      og = time.perf_counter()
      legality_list.append(legality)
      det_list.append(det_count)
      legality = 0 
      det_count = 1
      minute_count += 1




<IPython.core.display.Javascript object>

<br>
<br>

#### 3.04 STREAMING DETECTION FROM PRE-RECORDED VIDEO.

- Running the cells below intake video files from the mounted Google drive, run object detection on them frame by frame, and save the resulting video with detections and bounding boxes overlaid.

In [ ]:
!./darknet detector demo data/obj.data cfg/new_custom-yolov4-tiny-detector.cfg cfg/new_custom-yolov4-tiny-detector_best.weights -dont_show /mydrive/yolov4-tiny/test_videos/craw_01.mp4 -i 0 -out_filename /mydrive/yolov4-tiny/test_videos/results/craw_01_results.avi 

Streaming output truncated to the last 5000 lines.

 cvWriteFrame 
Objects:

CRAWFISH: 98% 
CRAWFISH: 77% 

FPS:36.6 	 AVG_FPS:36.3

 cvWriteFrame 
Objects:

CRAWFISH: 99% 
CRAWFISH: 78% 

FPS:35.2 	 AVG_FPS:36.3

 cvWriteFrame 
Objects:

CRAWFISH: 99% 
CRAWFISH: 95% 

FPS:34.3 	 AVG_FPS:38.2

 cvWriteFrame 
Objects:

CRAWFISH: 98% 
CRAWFISH: 92% 

FPS:33.9 	 AVG_FPS:38.2

 cvWriteFrame 
Objects:

CRAWFISH: 97% 
CRAWFISH: 93% 

FPS:33.4 	 AVG_FPS:38.2

 cvWriteFrame 
Objects:

CRAWFISH: 97% 
CRAWFISH: 94% 

FPS:32.4 	 AVG_FPS:38.2

 cvWriteFrame 
Objects:

CRAWFISH: 95% 
CRAWFISH: 94% 
CRAWFISH: 33% 

FPS:33.4 	 AVG_FPS:38.2

 cvWriteFrame 
Objects:

CRAWFISH: 95% 
CRAWFISH: 91% 

FPS:33.5 	 AVG_FPS:38.2

 cvWriteFrame 
Objects:

CRAWFISH: 98% 
CRAWFISH: 86% 
CRAWFISH: 47% 

FPS:37.2 	 AVG_FPS:38.2

 cvWriteFrame 
Objects:

CRAWFISH: 96% 
CRAWFISH: 94% 

FPS:37.4 	 AVG_FPS:38.2

 cvWriteFrame 
Objects:

CRAWFISH: 97% 
CRAWFISH: 92% 

FPS:36.4 	 AVG_FPS:38.2

 cvWriteFrame 
Objects:

CR

In [ ]:
!./darknet detector demo data/obj.data cfg/new_custom-yolov4-tiny-detector.cfg cfg/new_custom-yolov4-tiny-detector_best.weights -dont_show /mydrive/yolov4-tiny/test_videos/craw_02.mp4 -i 0 -out_filename /mydrive/yolov4-tiny/test_videos/results/craw_02_results.avi 

Streaming output truncated to the last 5000 lines.
 cvWriteFrame 
Objects:

CRAWFISH: 100% 
CRAWFISH: 100% 

FPS:45.8 	 AVG_FPS:37.2

 cvWriteFrame 
Objects:

CRAWFISH: 100% 
CRAWFISH: 100% 

FPS:46.2 	 AVG_FPS:37.2

 cvWriteFrame 
Objects:

CRAWFISH: 100% 
CRAWFISH: 100% 

FPS:47.9 	 AVG_FPS:37.2

 cvWriteFrame 
Objects:

CRAWFISH: 100% 
CRAWFISH: 100% 
CRAWFISH: 37% 

FPS:48.2 	 AVG_FPS:37.2

 cvWriteFrame 
Objects:

CRAWFISH: 100% 
CRAWFISH: 99% 

FPS:49.6 	 AVG_FPS:37.2

 cvWriteFrame 
Objects:

CRAWFISH: 100% 
CRAWFISH: 99% 
CRAWFISH: 35% 

FPS:49.7 	 AVG_FPS:37.2

 cvWriteFrame 
Objects:

CRAWFISH: 100% 
CRAWFISH: 94% 
CRAWFISH: 39% 

FPS:49.6 	 AVG_FPS:37.2

 cvWriteFrame 
Objects:

CRAWFISH: 100% 
CRAWFISH: 95% 
CRAWFISH: 58% 

FPS:47.2 	 AVG_FPS:37.2

 cvWriteFrame 
Objects:

CRAWFISH: 100% 
CRAWFISH: 97% 
CRAWFISH: 79% 

FPS:46.1 	 AVG_FPS:37.2

 cvWriteFrame 
Objects:

CRAWFISH: 99% 
CRAWFISH: 94% 
CRAWFISH: 54% 

FPS:44.5 	 AVG_FPS:37.2

 cvWriteFrame 
Objects:

CRAWFISH: 9

In [ ]:
!./darknet detector demo data/obj.data cfg/new_custom-yolov4-tiny-detector.cfg cfg/new_custom-yolov4-tiny-detector_best.weights -dont_show /mydrive/yolov4-tiny/test_videos/craw_03.mp4 -i 0 -out_filename /mydrive/yolov4-tiny/test_videos/results/craw_03_results.avi 

Streaming output truncated to the last 5000 lines.

 cvWriteFrame 
Objects:

CRAWFISH: 100% 
CRAWFISH: 90% 
CRAWFISH: 86% 

FPS:34.3 	 AVG_FPS:35.3

 cvWriteFrame 
Objects:

CRAWFISH: 100% 
CRAWFISH: 97% 
CRAWFISH: 89% 

FPS:36.4 	 AVG_FPS:35.3

 cvWriteFrame 
Objects:

CRAWFISH: 100% 
CRAWFISH: 98% 
CRAWFISH: 94% 

FPS:38.7 	 AVG_FPS:35.3

 cvWriteFrame 
Objects:

CRAWFISH: 100% 
CRAWFISH: 96% 
CRAWFISH: 93% 

FPS:40.8 	 AVG_FPS:35.3

 cvWriteFrame 
Objects:

CRAWFISH: 100% 
CRAWFISH: 98% 
CRAWFISH: 93% 

FPS:41.9 	 AVG_FPS:35.3

 cvWriteFrame 
Objects:

CRAWFISH: 100% 
CRAWFISH: 97% 
CRAWFISH: 89% 

FPS:41.8 	 AVG_FPS:35.3

 cvWriteFrame 
Objects:

CRAWFISH: 100% 
CRAWFISH: 98% 
CRAWFISH: 91% 

FPS:40.7 	 AVG_FPS:35.3

 cvWriteFrame 
Objects:

CRAWFISH: 100% 
CRAWFISH: 96% 
CRAWFISH: 96% 

FPS:40.7 	 AVG_FPS:35.3

 cvWriteFrame 
Objects:

CRAWFISH: 100% 
CRAWFISH: 98% 
CRAWFISH: 96% 

FPS:42.3 	 AVG_FPS:35.3

 cvWriteFrame 
Objects:

CRAWFISH: 100% 
CRAWFISH: 99% 
CRAWFISH: 95% 

FPS

In [ ]:
!./darknet detector demo data/obj.data cfg/new_custom-yolov4-tiny-detector.cfg cfg/new_custom-yolov4-tiny-detector_best.weights -dont_show /mydrive/yolov4-tiny/test_videos/craw_04.mp4 -i 0 -out_filename /mydrive/yolov4-tiny/test_videos/results/craw_04_results.avi 

Streaming output truncated to the last 5000 lines.
FPS:45.0 	 AVG_FPS:34.6

 cvWriteFrame 
Objects:

CRAWFISH: 98% 
CRAWFISH: 98% 
CRAWFISH: 60% 

FPS:43.5 	 AVG_FPS:34.6

 cvWriteFrame 
Objects:

CRAWFISH: 99% 
CRAWFISH: 98% 
CRAWFISH: 48% 

FPS:42.0 	 AVG_FPS:34.6

 cvWriteFrame 
Objects:

CRAWFISH: 100% 
CRAWFISH: 99% 
CRAWFISH: 48% 

FPS:41.0 	 AVG_FPS:34.6

 cvWriteFrame 
Objects:

CRAWFISH: 99% 
CRAWFISH: 97% 
CRAWFISH: 46% 

FPS:39.6 	 AVG_FPS:34.6

 cvWriteFrame 
Objects:

CRAWFISH: 100% 
CRAWFISH: 96% 
CRAWFISH: 54% 

FPS:38.0 	 AVG_FPS:34.6

 cvWriteFrame 
Objects:

CRAWFISH: 99% 
CRAWFISH: 96% 
CRAWFISH: 60% 

FPS:37.1 	 AVG_FPS:34.6

 cvWriteFrame 
Objects:

CRAWFISH: 100% 
CRAWFISH: 97% 
CRAWFISH: 49% 

FPS:37.0 	 AVG_FPS:34.6

 cvWriteFrame 
Objects:

CRAWFISH: 100% 
CRAWFISH: 98% 
CRAWFISH: 53% 

FPS:37.9 	 AVG_FPS:34.6

 cvWriteFrame 
Objects:

CRAWFISH: 100% 
CRAWFISH: 98% 
CRAWFISH: 50% 

FPS:39.8 	 AVG_FPS:34.6

 cvWriteFrame 
Objects:

CRAWFISH: 100% 
CRAWFISH: 97% 

In [ ]:
!./darknet detector demo data/obj.data cfg/new_custom-yolov4-tiny-detector.cfg cfg/new_custom-yolov4-tiny-detector_best.weights -dont_show /mydrive/yolov4-tiny/test_videos/craw_05.mp4 -i 0 -out_filename /mydrive/yolov4-tiny/test_videos/results/craw_05_results.avi 

Streaming output truncated to the last 5000 lines.
CRAWFISH: 97% 

FPS:47.2 	 AVG_FPS:36.6

 cvWriteFrame 
Objects:

CRAWFISH: 100% 
CRAWFISH: 92% 

FPS:47.7 	 AVG_FPS:36.6

 cvWriteFrame 
Objects:

CRAWFISH: 98% 
CRAWFISH: 96% 

FPS:49.0 	 AVG_FPS:36.6

 cvWriteFrame 
Objects:

CRAWFISH: 98% 
CRAWFISH: 95% 

FPS:46.7 	 AVG_FPS:36.6

 cvWriteFrame 
Objects:

CRAWFISH: 96% 
CRAWFISH: 95% 
CRAWFISH: 76% 

FPS:45.4 	 AVG_FPS:36.6

 cvWriteFrame 
Objects:

CRAWFISH: 97% 
CRAWFISH: 97% 
CRAWFISH: 92% 

FPS:43.5 	 AVG_FPS:36.6

 cvWriteFrame 
Objects:

CRAWFISH: 96% 
CRAWFISH: 90% 
CRAWFISH: 85% 

FPS:41.8 	 AVG_FPS:36.6

 cvWriteFrame 
Objects:

CRAWFISH: 95% 
CRAWFISH: 74% 
CRAWFISH: 63% 

FPS:40.8 	 AVG_FPS:36.6

 cvWriteFrame 
Objects:

CRAWFISH: 95% 
CRAWFISH: 63% 
CRAWFISH: 52% 

FPS:39.7 	 AVG_FPS:36.6

 cvWriteFrame 
Objects:

CRAWFISH: 100% 

FPS:38.3 	 AVG_FPS:36.6

 cvWriteFrame 
Objects:

CRAWFISH: 100% 
CRAWFISH: 52% 
CRAWFISH: 48% 

FPS:38.6 	 AVG_FPS:36.6

 cvWriteFrame 
Objec

In [ ]:
!./darknet detector demo data/obj.data cfg/new_custom-yolov4-tiny-detector.cfg cfg/new_custom-yolov4-tiny-detector_best.weights -dont_show /mydrive/yolov4-tiny/test_videos/craw_06.mp4 -i 0 -out_filename /mydrive/yolov4-tiny/test_videos/results/craw_06_results.avi 

Streaming output truncated to the last 5000 lines.
CRAWFISH: 100% 
CRAWFISH: 99% 
CRAWFISH: 90% 

FPS:40.2 	 AVG_FPS:36.9

 cvWriteFrame 
Objects:

CRAWFISH: 100% 
CRAWFISH: 99% 
CRAWFISH: 93% 

FPS:39.7 	 AVG_FPS:36.9

 cvWriteFrame 
Objects:

CRAWFISH: 100% 
CRAWFISH: 100% 
CRAWFISH: 93% 

FPS:39.5 	 AVG_FPS:36.9

 cvWriteFrame 
Objects:

CRAWFISH: 99% 
CRAWFISH: 99% 
CRAWFISH: 91% 

FPS:37.9 	 AVG_FPS:36.9

 cvWriteFrame 
Objects:

CRAWFISH: 100% 
CRAWFISH: 99% 
CRAWFISH: 91% 

FPS:39.0 	 AVG_FPS:36.9

 cvWriteFrame 
Objects:

CRAWFISH: 99% 
CRAWFISH: 99% 
CRAWFISH: 85% 

FPS:41.6 	 AVG_FPS:36.9

 cvWriteFrame 
Objects:

CRAWFISH: 100% 
CRAWFISH: 99% 
CRAWFISH: 86% 

FPS:43.2 	 AVG_FPS:36.9

 cvWriteFrame 
Objects:

CRAWFISH: 100% 
CRAWFISH: 100% 
CRAWFISH: 96% 

FPS:44.4 	 AVG_FPS:36.9

 cvWriteFrame 
Objects:

CRAWFISH: 99% 
CRAWFISH: 99% 
CRAWFISH: 97% 

FPS:44.3 	 AVG_FPS:36.9

 cvWriteFrame 
Objects:

CRAWFISH: 100% 
CRAWFISH: 99% 
CRAWFISH: 97% 

FPS:42.4 	 AVG_FPS:36.9

 cvWr